In [1]:
from langchain_community.document_loaders import JSONLoader
from langchain_google_vertexai import VertexAI,VertexAIEmbeddings,ChatVertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import json,pandas


2024-03-13 12:02:56.064964: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 12:02:56.897152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 12:02:56.897266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 12:02:57.082832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 12:02:57.458630: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 12:02:57.463553: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:

embeddings=VertexAIEmbeddings(project="langchain-genai")

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


PermissionDenied: 403 Your application is authenticating by using local Application Default Credentials. The aiplatform.googleapis.com API requires a quota project, which is not set by default. To learn how to set your quota project, see https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds . [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "consumer"
  value: "projects/618104708054"
}
]

# ADDING PRODUCT ID TO DOCUMENTS TO MATCH WITH CATALOG

In [28]:
df=pandas.read_json("catalog.json")

In [30]:
product_ids=[f"product-{i}" for i in range(101,295)]

In [31]:
df["product_id"]=product_ids

In [34]:
data={"data":df.to_dict(orient="records")}

In [40]:
file=open("./catalog-new.json","w")
file.write(json.dumps(data))

116001

#   CREATING VECTOR STORE BY CREATING EMBEDDINGS 

In [41]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["id"]=record.get("product_id")
    metadata["product_name"] = record.get("product_name")
    metadata["price"] = record.get("price")
    metadata["category"]=record.get("category")

    return metadata
loader = JSONLoader(
    file_path='./catalog-new.json',
 jq_schema='.data[]',
    content_key="description",
    metadata_func=metadata_func
)

data = loader.load()

In [42]:
data

[Document(page_content="This stylish JACK&JONES formal shirt is a must-have for your wardrobe. Crafted from premium cotton blend fabric, it's comfortable and perfect for any formal occasion. Wear it with a pair of jeans and boots for a casual yet cool look, or dress it up with chinos and loafers to make an impression. This is a formal shirt for men.", metadata={'source': '/mnt/c/Users/gudik/Downloads/langchain/catalog-new.json', 'seq_num': 1, 'id': 'product-101', 'product_name': 'Black Formal Full Sleeves Shirt', 'price': 999, 'category': 'shirts'}),
 Document(page_content='Elevate your style game with the epitome of print-blocking finesse – the check-printed cut & sew shirt from the UNMATCHED by JACK&JONES collection. Crafted from premium 100% cotton fabric, this full sleeve shirt boasts an edgy enzyme wash, delivering a classic regular fit with a contemporary twist. Pair it effortlessly with slightly distressed jeans and your favourite kicks for a seamlessly smart, casual, and undeni

In [45]:
split=RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [46]:
docs=split.split_documents(data)

In [50]:
chroma=Chroma.from_documents(embedding=embeddings,documents=docs,persist_directory="./chromadb")

In [51]:
chroma.similarity_search("sun")

[Document(page_content="Elevate your chill vibes with this insanely cool shirt from UNMATCHED by JACK&JONES! Crafted from 100% cotton, this full-sleeve shirt for men brings the ultimate combo of slick stripes and solid print together. Sporting a drop oversized fit, it's not just a shirt—it's a statement. Perfect for flexin' as an overshirt or layering up for those dope streetstyle moments. Pair it with your fave jeans and kicks, and you've got a look that screams street smart!", metadata={'category': 'shirts', 'id': 'product-105', 'price': 6999, 'product_name': 'UNMATCHED by JACK&JONES Black Print-Blocked Cut & Sew Shirt', 'seq_num': 5, 'source': '/mnt/c/Users/gudik/Downloads/langchain/catalog-new.json'}),
 Document(page_content="Men's 100% cotton full sleeves shirt rendered in an all over print. Offers a slim fit and is designed with classic front button closure. Perfect to wear with trousers and formal shoes for a smart-casual look.", metadata={'category': 'shirts', 'id': 'product-12